In [1]:
import pandas as pd
import re
import random
import numpy as np

In [2]:
def get_df_norm_header_index(path):
    lst_header = ['num_exp', 'dia', 'hora', 'calle', 'num', 'distrito', 'tipo_accidente', 'meteo', 'vehiculo', 'persona', 'edad', 'sexo', 'lesividad' ]
    df = pd.read_csv(path, encoding='latin_1',sep = ';', names=lst_header)
    df.drop(0, inplace=True)
    df.reset_index(drop=True, inplace=True)
    return df

bici = get_df_norm_header_index('bicicletas/AccidentesBicicletas_2019.csv')

In [3]:
bici.dtypes

num_exp           object
dia               object
hora              object
calle             object
num               object
distrito          object
tipo_accidente    object
meteo             object
vehiculo          object
persona           object
edad              object
sexo              object
lesividad         object
dtype: object

In [4]:
def create_date(df):
    df['fecha'] = df.dia+' '+df.hora
    bici['fecha'] = pd.to_datetime(df['fecha'], dayfirst=True)
    df.drop(['dia', 'hora'], axis=1, inplace = True)
    return df

bici = create_date(bici)

In [5]:
def create_dir(df):
    df.num.apply(lambda x: x.strip())
    df.loc[(df.num == '-'), 'num']=''
    df['direccion'] = df.calle+' '+df.num
    df.direccion.apply(lambda x: x.strip())
    df.drop(['calle', 'num'], axis=1, inplace = True)
    return df

bici = create_dir(bici)

In [6]:
bici.head()

,num_exp,distrito,tipo_accidente,meteo,vehiculo,persona,edad,sexo,lesividad,fecha,direccion
0,2019S000659,SALAMANCA,Alcance,Despejado,Bicicleta,Conductor,DE 25 A 29 AÑOS,Hombre,01,2019-01-01 14:00:00,CALL. CASTELLO / CALL. DON RAMON DE LA CRUZ
1,2019S000036,HORTALEZA,Colisión fronto-lateral,Despejado,Bicicleta,Conductor,DE 70 A 74 AÑOS,Hombre,03,2019-01-02 20:45:00,AVDA. GRAN VIA DE HORTALEZA / GTA. LUIS ROSALES
2,2019S000133,VILLA DE VALLECAS,Alcance,Se desconoce,Bicicleta,Conductor,DE 15 A 17 AÑOS,Hombre,07,2019-01-03 14:30:00,CALL. FELIPE ALVAREZ 10
3,2019S000132,VILLA DE VALLECAS,Alcance,Despejado,Bicicleta,Conductor,DE 18 A 20 AÑOS,Hombre,07,2019-01-03 12:45:00,AVDA. SANTA EUGENIA / CALL. REAL DE ARGANDA
4,2019S000132,VILLA DE VALLECAS,Alcance,Despejado,Bicicleta,Conductor,DE 21 A 24 AÑOS,Hombre,14,2019-01-03 12:45:00,AVDA. SANTA EUGENIA / CALL. REAL DE ARGANDA


In [7]:
bici.tipo_accidente.isna().sum()

0

In [8]:
def change_accident(values):
    dict_accidente = {
        r'Caída':'Caida',
        r'Colisión':'Colision',
        r'Alcance':'Colision',
        r'Atropello':'Imprudencia',
        r'Choque':'Imprudencia'}
    for k, v in dict_accidente.items():
        if re.match(k, values):
            return v
    return 'Otro'

bici.tipo_accidente = bici.tipo_accidente.apply(lambda x: change_accident(x))

In [9]:
bici.tipo_accidente.value_counts(dropna= False)

Colision       388
Caida          230
Imprudencia    141
Otro            20
Name: tipo_accidente, dtype: int64

In [10]:
bici.lesividad.value_counts(dropna=False)

07     275
02     120
14      98
06      83
NaN     80
01      66
05      35
03      21
04       1
Name: lesividad, dtype: int64

In [11]:
##
#Desde aqui
##
bici[bici.lesividad.isna()].tipo_accidente.value_counts()

Colision       54
Imprudencia    12
Caida          11
Otro            3
Name: tipo_accidente, dtype: int64

In [12]:
aux = list(set(bici[bici.lesividad.isna()].tipo_accidente))
aux

['Otro', 'Caida', 'Colision', 'Imprudencia']

In [13]:
bici[bici.tipo_accidente=='Colision'].lesividad.value_counts(normalize = True)

07    0.446108
02    0.155689
14    0.131737
06    0.089820
01    0.074850
05    0.062874
03    0.035928
04    0.002994
Name: lesividad, dtype: float64

In [14]:
dict_acc_lesi = dict(bici[bici.tipo_accidente=='Colision'].lesividad.value_counts(normalize = True))
new_v = list(np.cumsum(list(dict_acc_lesi.values())))
keys = list(dict_acc_lesi.keys())
dict_acc_lesi = {round(v,2):k for k,v in zip(keys, new_v)}  
dict_acc_lesi

{0.45: '07',
 0.6: '02',
 0.73: '14',
 0.82: '06',
 0.9: '01',
 0.96: '05',
 1.0: '04'}

In [18]:
# def coef_injury(df):
#     lst_injury_fill = list(set(df[df.lesividad.isna()].tipo_accidente))
#     for e in lst_injury_fill:
#         dict_acc_lesi = dict(df[df.tipo_accidente==e].lesividad.value_counts(normalize = True))
#         new_v = list(np.cumsum(list(dict_acc_lesi.values())))
#         keys = list(dict_acc_lesi.keys())
#         dict_acc_lesi = {round(v,2):k for k,v in zip(keys, new_v)}  
#         yield dict_acc_lesi       


def fill_injury(values, df):
    lst_injury_fill = list(set(df[df.lesividad.isna()].tipo_accidente))
    for e in lst_injury_fill:
        dal = dict(df[df.tipo_accidente==e].lesividad.value_counts(normalize = True))
        new_v = list(np.cumsum(list(dal.values())))
        keys = list(dal.keys())
        dal = {round(v,2):k for k,v in zip(keys, new_v)}
        

        rand = random.random()
        for k,v in dal.items():
            if values < k:
                return v

bici.lesividad = bici.lesividad.apply(lambda x: fill_injury(x, bici))

TypeError: '>' not supported between instances of 'numpy.ndarray' and 'str'

In [15]:
bici[bici.tipo_accidente=='Colision'].lesividad.value_counts(normalize = True)

0.45
0.6
0.73
0.82
0.9
0.96
1.0


In [15]:
def change_injury(values):
    dict_injury = {
        '01':'Moderada','02':'Moderada',
        '03':'Grave',
        '04':'Fallecido',
        '05':'Leve','06':'Leve','07':'Leve','14':'Leve'}
    try: 
        return dict_injury[values]
    except:
        pass

bici.lesividad = bici.lesividad.apply(lambda x: change_injury(str(x)))

In [18]:
bici.head()

,num_exp,distrito,tipo_accidente,meteo,vehiculo,persona,edad,sexo,lesividad,fecha,direccion
0,2019S000659,SALAMANCA,Colision,Despejado,Bicicleta,Conductor,DE 25 A 29 AÑOS,Hombre,None,2019-01-01 14:00:00,CALL. CASTELLO / CALL. DON RAMON DE LA CRUZ
1,2019S000036,HORTALEZA,Colision,Despejado,Bicicleta,Conductor,DE 70 A 74 AÑOS,Hombre,None,2019-01-02 20:45:00,AVDA. GRAN VIA DE HORTALEZA / GTA. LUIS ROSALES
2,2019S000133,VILLA DE VALLECAS,Colision,Se desconoce,Bicicleta,Conductor,DE 15 A 17 AÑOS,Hombre,None,2019-01-03 14:30:00,CALL. FELIPE ALVAREZ 10
3,2019S000132,VILLA DE VALLECAS,Colision,Despejado,Bicicleta,Conductor,DE 18 A 20 AÑOS,Hombre,None,2019-01-03 12:45:00,AVDA. SANTA EUGENIA / CALL. REAL DE ARGANDA
4,2019S000132,VILLA DE VALLECAS,Colision,Despejado,Bicicleta,Conductor,DE 21 A 24 AÑOS,Hombre,None,2019-01-03 12:45:00,AVDA. SANTA EUGENIA / CALL. REAL DE ARGANDA


In [14]:
bici[bici.tipo_accidente == 'Otro'].lesividad.value_counts()

Series([], Name: lesividad, dtype: int64)

In [22]:
bici[bici.lesividad.isna()].tipo_accidente.value_counts()

Alcance                         20
Colisión fronto-lateral         18
Colisión lateral                12
Caída                           11
Atropello a persona              9
Colisión frontal                 3
Choque contra obstáculo fijo     3
Otro                             3
Colisión múltiple                1
Name: tipo_accidente, dtype: int64

In [11]:
bici.persona.value_counts()

Conductor    707
Peatón        68
Pasajero       4
Name: persona, dtype: int64

In [12]:
bici.sexo.value_counts()

Hombre    574
Mujer     192
Name: sexo, dtype: int64

In [ ]:
bici.drop(['sexo', 'vehiculo'], axis=1, inplace = True)

In [ ]:
# encoding='cp850'
# encoding='cp1250'

# **
# encoding='latin_1'
# encoding='iso8859_15'